In [ ]:
import misc
import holodeck as holo
import numpy as np
import matplotlib.pyplot as plt
from holodeck import utils
from holodeck import plot as hplot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, MYR
import gwb as psgwb
import seaborn as sns

# Generate evolution objects for populations with and without CBD influence

In [ ]:
keys = ['no_doteb', 'doteb']
f_edd = 0.1
nsteps = 100
eccen_init=0.01
keys, evol_dict = misc.generate_evol(keys, f_edd=f_edd, eccen_init=eccen_init, ecc_test=True, nsteps=nsteps)

In [ ]:
fig, ax = holo.plot.figax()
ax.grid(True, alpha=0.25)
fobs, _ = utils.pta_freqs(num=40)
nreals=30
plot_nanograv23 = True

lw = 3
ticksize = 5
tickwidth = 2
alpha = 1.0
fs = 20

for key in keys:
    evol = evol_dict[key]
    gwb = holo.gravwaves.GW_Discrete(evol, fobs, nreals=nreals)
    gwb.emit(eccen=True)
    median_gwb = np.median(gwb.both, axis=-1)

    if 'ls_%s' %key in evol_dict.keys():
        ls = evol_dict['ls_%s' %key]
    else:
        ls = '-'

    cc, = ax.plot(fobs, median_gwb, label=evol_dict['label_%s' %key], \
                    color=evol_dict['color_%s' %key], linewidth=lw, \
                    linestyle=ls)
    conf = np.percentile(gwb.both, [25, 75], axis=-1)
    ax.fill_between(fobs, *conf, color=cc.get_color(), alpha=0.1)

    twin_ax = hplot._twin_yr(ax, nano=False, fs=fs, label=False)

    plt.setp(twin_ax.get_xticklabels(which='both'), fontsize=fs, rotation=0)
    plt.setp(twin_ax.get_yticklabels(which='both'), fontsize=fs)
    twin_ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    twin_ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)

    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=0)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)

if plot_nanograv23:
    f_det = 1./YR
    amp_det = 2.4*10**(-15)
    err_det = np.array([[0.6*10**(-15),0.7*10**(-15)]]).T
    ax.errorbar(f_det, amp_det, yerr=err_det, color = 'royalblue', \
            marker='*', markersize=4*lw, mew=lw, label='NANOGrav\n(2023)')

twin_ax.set_xlabel(r'$f_{\rm GW} \ [\rm{yr}^{-1}]$', fontsize=fs)
ax.set_xlabel(r'$f_{\rm GW} \ [\rm{Hz}]$', fontsize=fs)
ax.legend(fontsize=0.8*fs, ncol=max(1,int(len(keys)/2.)+1), loc='lower center')
fig.tight_layout()

# What causes the high amplitude in the CBD case? Maybe eccentric harmonics?

In [ ]:
fig, ax = holo.plot.figax()
ax.grid(True, alpha=0.25)
fobs, _ = utils.pta_freqs(num=40)
nreals=30
plot_nanograv23 = True

lw = 3
ticksize = 5
tickwidth = 2
alpha = 1.0
fs = 20

key = 'doteb'
evol = evol_dict[key]
gwb = holo.gravwaves.GW_Discrete(evol, fobs, nreals=nreals)

#now compare cases where eccen = True and eccen = False
for eccen,label in zip([False,True],['eccen (circ-GWB)','eccen GWB']):
    gwb.emit(eccen=eccen)
    median_gwb = np.median(gwb.both, axis=-1)

    cc, = ax.plot(fobs, median_gwb, label=label, \
                    linewidth=lw, \
                    linestyle=ls)
    conf = np.percentile(gwb.both, [25, 75], axis=-1)
    ax.fill_between(fobs, *conf, color=cc.get_color(), alpha=0.1)

twin_ax = hplot._twin_yr(ax, nano=False, fs=fs, label=False)

plt.setp(twin_ax.get_xticklabels(which='both'), fontsize=fs, rotation=0)
plt.setp(twin_ax.get_yticklabels(which='both'), fontsize=fs)
twin_ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
twin_ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)

plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=0)
plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)

if plot_nanograv23:
    f_det = 1./YR
    amp_det = 2.4*10**(-15)
    err_det = np.array([[0.6*10**(-15),0.7*10**(-15)]]).T
    ax.errorbar(f_det, amp_det, yerr=err_det, color = 'royalblue', \
            marker='*', markersize=4*lw, mew=lw, label='NANOGrav\n(2023)')

twin_ax.set_xlabel(r'$f_{\rm GW} \ [\rm{yr}^{-1}]$', fontsize=fs)
ax.set_xlabel(r'$f_{\rm GW} \ [\rm{Hz}]$', fontsize=fs)
ax.legend(fontsize=0.8*fs, ncol=max(1,int(len(keys)/2.)+1), loc='lower center')
fig.tight_layout()

# Could initial eccentricity change the results?

In [ ]:
keys = ['doteb', 'no_doteb']
key_labels = ['CBD', 'no CBD']
keys_ecc = []
evol_dict_ecc = {}
all_ls = []
all_eccen = [1.e-4, 0.01, 0.1, 0.9]
for eccen_init,color in zip(all_eccen,sns.color_palette('colorblind')):
    key_ecc = 'eb0=%.4f' %eccen_init
    keys_e, evol_dict_e = misc.generate_evol(keys, f_edd=f_edd, eccen_init=eccen_init, \
                                        ecc_test=True, nsteps=nsteps)
    linestyles = ['-', ':']
    for i in range(0,len(keys_e)):
        key_ecc_i = keys_e[i] + '_' + key_ecc
        evol_dict_ecc['label_%s' %key_ecc_i] = r'$e_{\rm b,0} = %.4f$' %eccen_init + '\n%s' %key_labels[i]
        evol_dict_ecc[key_ecc_i] = evol_dict_e[keys_e[i]]
        evol_dict_ecc['color_%s' %key_ecc_i] = color
        evol_dict_ecc['ls_%s' %key_ecc_i] = linestyles[i]
        keys_ecc.append(key_ecc_i)
        evol_dict_ecc[key_ecc_i] = evol_dict_e[keys_e[i]]
        all_ls.append(linestyles[i])


In [ ]:
#Then plot the spectra of all the above
psgwb.plot(keys_ecc, evol_dict_ecc, savefig=False)

# Does accretion/mass ratio growth drive this change? Let's see what happens if we compare the following cases:
## - no CBD accretion, no CBD torques
## - CBD accretion, no CBD torques
## - no CBD accretion, CBD torques
## - CBD accretion AND CBD torques

In [ ]:
import misc
import holodeck as holo
import numpy as np
import matplotlib.pyplot as plt
from holodeck import utils
from holodeck import plot as hplot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, MYR
keys = ['no_doteb', 'acc_no_doteb', 'no_acc_doteb', 'doteb']
labels = ['nT+nA', 'nT+A', 'T+nA', 'T+A']
f_edd = 0.1
nsteps = 100
eccen_init=0.01
keys, evol_dict = misc.generate_evol(keys, f_edd=f_edd, eccen_init=eccen_init, \
                                   ecc_test=True, nsteps=nsteps, labels=labels)


In [ ]:
fig, ax = holo.plot.figax()
ax.grid(True, alpha=0.25)
fobs, _ = utils.pta_freqs(num=40)
nreals=30
plot_nanograv23 = True

lw = 3
ticksize = 5
tickwidth = 2
alpha = 1.0
fs = 20

for key in keys:
    evol = evol_dict[key]
    gwb = holo.gravwaves.GW_Discrete(evol, fobs, nreals=nreals)
    gwb.emit(eccen=True)
    median_gwb = np.median(gwb.both, axis=-1)

    if 'ls_%s' %key in evol_dict.keys():
        ls = evol_dict['ls_%s' %key]
    else:
        ls = '-'

    cc, = ax.plot(fobs, median_gwb, label=evol_dict['label_%s' %key], \
                    color=evol_dict['color_%s' %key], linewidth=lw, \
                    linestyle=ls)
    conf = np.percentile(gwb.both, [25, 75], axis=-1)
    ax.fill_between(fobs, *conf, color=cc.get_color(), alpha=0.1)

    twin_ax = hplot._twin_yr(ax, nano=False, fs=fs, label=False)

    plt.setp(twin_ax.get_xticklabels(which='both'), fontsize=fs, rotation=0)
    plt.setp(twin_ax.get_yticklabels(which='both'), fontsize=fs)
    twin_ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    twin_ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)

    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=0)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)

if plot_nanograv23:
    f_det = 1./YR
    amp_det = 2.4*10**(-15)
    err_det = np.array([[0.6*10**(-15),0.7*10**(-15)]]).T
    ax.errorbar(f_det, amp_det, yerr=err_det, color = 'royalblue', \
            marker='*', markersize=4*lw, mew=lw, label='NANOGrav\n(2023)')

twin_ax.set_xlabel(r'$f_{\rm GW} \ [\rm{yr}^{-1}]$', fontsize=fs)
ax.set_xlabel(r'$f_{\rm GW} \ [\rm{Hz}]$', fontsize=fs)
ax.legend(fontsize=0.8*fs, ncol=max(1,int(len(keys)/2.)+1), loc='lower center')
fig.tight_layout()

In [ ]:
import pdf_xt_pop as pfx
fobs_orb_edges = np.logspace(-9,0,1000)
window_param = 'fobs'
freq_windows = [[1.e-9,1.e-8]]
windows = freq_windows
window_labels = ['PTA']

params = ['eccen', 'mrat']
all_param_bins = [np.logspace(-3,0,30),np.logspace(-1,0,15)]
pfx.plot(keys, evol_dict, params, all_param_bins, fobs_orb_edges, \
         window_param=window_param, windows=windows, \
         window_labels=window_labels, all_in_one_ax=False, \
         show_median=True, \
         savefig=False)

params = ['mtot', 'mchirp']
all_param_bins = [np.logspace(6,10,20)*MSOL, np.logspace(6,10,20)*MSOL]
pfx.plot(keys, evol_dict, params, all_param_bins, fobs_orb_edges, \
         window_param=window_param, windows=windows, \
         window_labels=window_labels, all_in_one_ax=False, \
         show_median=True, \
         savefig=False)

In [ ]:
fobs_orb_edges = np.logspace(-5,0,1000)
window_param = 'fobs'
freq_windows = [[1.e-4,1.0]]
windows = freq_windows
window_labels = ['LISA']

params = ['eccen', 'mrat']
all_param_bins = [np.logspace(-7,0,30),np.logspace(-1,0,15)]
pfx.plot(keys, evol_dict, params, all_param_bins, fobs_orb_edges, \
         window_param=window_param, windows=windows, \
         window_labels=window_labels, all_in_one_ax=False, \
         show_median=True, \
         savefig=False)

params = ['mtot', 'mchirp']
all_param_bins = [np.logspace(6,10,20)*MSOL, np.logspace(6,10,20)*MSOL]
pfx.plot(keys, evol_dict, params, all_param_bins, fobs_orb_edges, \
         window_param=window_param, windows=windows, \
         window_labels=window_labels, all_in_one_ax=False, \
         show_median=True, \
         savefig=False)

# Now let's see how accretion rate affects this

In [ ]:
import holodeck as holo
import numpy as np
import matplotlib.pyplot as plt
from holodeck import utils
from holodeck import plot as hplot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, MYR
keys = ['no_doteb', 'acc_no_doteb', 'no_acc_doteb', 'doteb']
labels = ['nT+nA', 'nT+A', 'T+nA', 'T+A']
f_edd = 0.1
nsteps = 100
eccen_init=0.01
f_edds = [1.e-3, 1.e-2, 0.05, 0.1, 0.5]

keys, evol_dict = misc.generate_evol(keys, f_edd=f_edd, eccen_init=eccen_init, \
                                   f_edd_test=True, nsteps=nsteps, f_edds=f_edds)

In [ ]:
figwidth = 10
figheight = 7
fig, ax = holo.plot.figax(figsize=[figwidth, figheight])
ax.grid(True, alpha=0.25)
fobs, _ = utils.pta_freqs(num=40)
nreals=30
plot_nanograv23 = True

lw = 3
ticksize = 5
tickwidth = 2
alpha = 1.0
fs = 20

plt.title("Compare Eddington fractions")

for key in keys:
    evol = evol_dict[key]
    gwb = holo.gravwaves.GW_Discrete(evol, fobs, nreals=nreals)
    gwb.emit(eccen=True)
    median_gwb = np.median(gwb.both, axis=-1)

    if 'ls_%s' %key in evol_dict.keys():
        ls = evol_dict['ls_%s' %key]
    else:
        ls = '-'

    cc, = ax.plot(fobs, median_gwb, label=evol_dict['label_%s' %key], \
                    color=evol_dict['color_%s' %key], linewidth=lw, \
                    linestyle=ls)
    conf = np.percentile(gwb.both, [25, 75], axis=-1)
    ax.fill_between(fobs, *conf, color=cc.get_color(), alpha=0.1)

    twin_ax = hplot._twin_yr(ax, nano=False, fs=fs, label=False)

    plt.setp(twin_ax.get_xticklabels(which='both'), fontsize=fs, rotation=0)
    plt.setp(twin_ax.get_yticklabels(which='both'), fontsize=fs)
    twin_ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    twin_ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)

    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=0)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)

if plot_nanograv23:
    f_det = 1./YR
    amp_det = 2.4*10**(-15)
    err_det = np.array([[0.6*10**(-15),0.7*10**(-15)]]).T
    ax.errorbar(f_det, amp_det, yerr=err_det, color = 'royalblue', \
            marker='*', markersize=4*lw, mew=lw, label='NANOGrav\n(2023)')

twin_ax.set_xlabel(r'$f_{\rm GW} \ [\rm{yr}^{-1}]$', fontsize=fs)
ax.set_xlabel(r'$f_{\rm GW} \ [\rm{Hz}]$', fontsize=fs)
ax.legend(fontsize=0.8*fs, ncol=max(1,int(len(keys)/2.)+1), loc='lower center')
fig.tight_layout()

In [ ]:
fobs_orb_edges = np.logspace(-9,0,1000)
window_param = 'fobs'
freq_windows = [[1.e-9,1.e-8]]
windows = freq_windows
window_labels = ['PTA']

params = ['eccen', 'mrat']
all_param_bins = [np.logspace(-3,0,30),np.logspace(-1,0,15)]
pfx.plot(keys, evol_dict, params, all_param_bins, fobs_orb_edges, \
         window_param=window_param, windows=windows, \
         window_labels=window_labels, all_in_one_ax=False, \
         show_median=True, \
         savefig=False)

params = ['mtot', 'mchirp']
all_param_bins = [np.logspace(6,10,20)*MSOL, np.logspace(6,10,20)*MSOL]
pfx.plot(keys, evol_dict, params, all_param_bins, fobs_orb_edges, \
         window_param=window_param, windows=windows, \
         window_labels=window_labels, all_in_one_ax=False, \
         show_median=True, \
         savefig=False)